In [413]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [415]:
df_v = pd.read_csv('data/volunteer.csv')
df_v.head(2)

,Name,Email,Phone Number,Canceled,Notes,VIP,Station,Day,EVENT LOCATION,Employer,"Title, Industry","City, State",Career Fields
0,Jiaxian Cai,NaN,NaN,False,NaN,False,Station 1,Wednesday,SJSU,Accenture,Application Support Analyst,"Mountain View,CA",NaN
1,Kevin Orlando Rodezno Polanco,NaN,NaN,False,NaN,False,Station 1,Wednesday,PayPal,Accenture,Management Consulting Senior Analyst,NaN,Business; Technology; Management;


In [416]:
df_s = pd.read_csv('data/student.csv')
df_s.head(2)

,Full Name,Student ID,First,Last,Cohort,Evening,Location,Phone Number,Email,Career Interests for Matching,Attendance,Module Score,Project Score,Bonus,Total Score
0,Adam Ababneh,NaN,Adam,Ababneh,SJSU Karin (Tu),Tuesday,SJSU,NaN,NaN,Business;,0.857143,0.815836,0.833333,NaN,251.0
1,Akhil Modali,NaN,Akhil,Modali,SJSU Allie (Tu),Tuesday,SJSU,NaN,NaN,Public Relations; Education;,1.000000,0.918337,0.916667,NaN,284.0


In [417]:
df_user = df_s[['Full Name', 'Career Interests for Matching']]
df_user["Career Interests for Matching"] = df_user["Career Interests for Matching"].fillna("Behavioral;")
df_user.head(2)

/Users/nasrajan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Full Name,Career Interests for Matching
0,Adam Ababneh,Business;
1,Akhil Modali,Public Relations; Education;


In [418]:
def removenulls(value):
    if value:
        return True
    else:
        return False
    
skills = df_user["Career Interests for Matching"].str.split(";").apply(lambda x: list(filter(removenulls, x)))
                                                
skills = skills.apply(lambda x: list(map(str.lstrip, x)) ) 
skills_list = [st.lstrip().rstrip() for row in skills for st in row]

df_user["interests"] = skills

df_user.head(2)

/Users/nasrajan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,Full Name,Career Interests for Matching,interests
0,Adam Ababneh,Business;,[Business]
1,Akhil Modali,Public Relations; Education;,"[Public Relations, Education]"


In [419]:
#df_user = df_user.assign(**pd.get_dummies(df_user.interests.apply(lambda x:pd.Series(x)).stack().reset_index(level=1,drop=True)).sum(level=0))
mlb = MultiLabelBinarizer()
df_user = df_user.join(pd.DataFrame(mlb.fit_transform(df_user.pop('interests')),
                       columns=mlb.classes_,
                          index=df_user.index))

df_user_profile = df_user.drop(columns=["Career Interests for Matching"])



In [420]:
df_v = df_v[['Name', 'Career Fields']]
df_v["Career Fields"] = df_v["Career Fields"].fillna("Behavioral;")
df_v.shape

(127, 2)

In [385]:
skills = df_user["Career Interests for Matching"].str.split(";").apply(lambda x: list(filter(removenulls, x)))
                                                
skills = skills.apply(lambda x: list(map(str.lstrip, x)) ) 

skills_list = [st.lstrip().rstrip() for row in skills for st in row]
df_v["interests"] = skills
df_v.head()

,Name,Career Fields,interests
0,Jiaxian Cai,Behavioral;,[Business]
1,Kevin Orlando Rodezno Polanco,Business; Technology; Management;,"[Public Relations, Education]"
2,Allison Meyers,Business; Management;,[Human Resources]
3,Suzette Sandoval,Human Resources; Recruiting;,[Behavioral]
4,Dionne Gayle,Business;,[Behavioral]


In [386]:
df_v = df_v.join(pd.DataFrame(mlb.fit_transform(df_v.pop('interests')),
                       columns=mlb.classes_,
                          index=df_v.index))

df_volunteer_profile = df_v.drop(columns=["Career Fields"])


In [387]:
df_volunteer_profile.columns.shape


(16,)

In [412]:
df_user_profile = df_user_profile.drop(columns=["Advertising", "Human Relations", "Public Rleations"])    

KeyError: "['Advertising' 'Human Relations' 'Public Rleations'] not found in axis"

In [398]:
for col in df_volunteer_profile.columns: 
    print(col) 

Name
Accounting
Behavioral
Business
Communications
Education
Engineering
Finance
Health
Human Resources
Journalism
Management
Marketing
Public Relations
Technology
Writing


In [399]:
df_user_profile.shape

(199, 16)

In [400]:
df_volunteer_profile = df_volunteer_profile.dropna(subset=['Name'])

In [401]:
X = df_volunteer_profile.drop('Name',1)
Y = df_volunteer_profile['Name']

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=42)

In [402]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=50, metric='cosine')
neigh.fit(X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
           metric_params=None, n_jobs=None, n_neighbors=50, p=2,
           weights='uniform')

In [403]:
from sklearn import metrics
Y_pred = neigh.predict(X_test)

# compare actual response values (y_test) with predicted response values (y_pred)
print(metrics.accuracy_score(Y_test, Y_pred))


0.0


In [404]:
import pickle
filename = 'knn.pickle'
pickle.dump(neigh, open(filename, 'wb'))

In [408]:
data = df_user_profile.drop(columns=['Full Name']).iloc[0]
loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.predict([data])
result = neigh.kneighbors([data])
print(result[1])

[[ 24  41   3  64  97  42  69  67  47  86  65  20  90  18  91  73  55  59
   62   4  98   7   5   9  60  95  61  14   8 100  75  88  99  89  92  93
   94  96  85  63  74  58  72  71  70  68  80  66  87  79]]


In [411]:
for i in result[1]:
    print(df_volunteer_profile.iloc[i]['Name'])

24               Ram Nagarajan
41              Chris Reynolds
3            Suzette Sandoval 
64                 Jay Staples
97                Monica Walls
42               Lindley Huynh
69                  Brad Mears
67                   Noha Rizk
47                 Chris Dedow
86              Cesar Zaragoza
65           Manickkam Pandian
20                 Siddhi Shah
90                  Peter Thai
18                 Katie Upton
91                Elliot Fiske
73              Rasana Anandan
55                Matt Sorlien
59               Ike Backstrom
62          Priya Balakrishnan
4                 Dionne Gayle
98          Sandra Escajadillo
7             Charles Anderson
5              Daman Ahluwalia
9              Jessica Mederos
60                 Julia David
95                  Amna Qamar
61          Sunethra Dhileepan
14                   Lucy Ring
8               Lauren Koehler
100            Deborah Jiminez
75                 Jing Osborn
88     Seyedshahin Ashrafzadeh
99      

In [144]:
for col in df_volunteer_profile.columns: 
    print(col) 

Name

 
 Accounting
 Business
 Communications
 Education
 Engineering
 English
 Finance
 Human Resources
 Management
 Marketing
 Public Relations
 Recruiting
 Technology
Behavioral
Business
Education
Engineering
Finance
Human Resources
Management
Marketing
Non-profit
Operations
Public Relations
Recruiting
Sales
Software Engineering
Technology


Full Name

 Accounting
 Business
 Communications
 Education
 Engineering
 Finance
 Health
 Human Relations
 Human Resources
 Journalism
 Management
 Marketing
 Public Relations
 Technology
 Writing
Accounting
Advertising
Behavioral
Business
Communications
Education
Engineering
Finance
Health
Human Resources
Management
Marketing
Public Relations
Public Rleations
Technology
Writing
